Introduction

In [ ]:
from datetime import datetime
import pytz

import matplotlib.pyplot as plt
import empyrical as ep
import pandas as pd
import pyfolio as pf
from trading.v2 import BacktestEngine
from trading.v2.util import read_data
from trading.v2.strategy import StrategyBase


class BuyAndHoldStrategy(StrategyBase):
    """
    buy on the first tick then hold to the end
    """

    def __init__(self):
        super(BuyAndHoldStrategy, self).__init__()
        self.invested = False

    def on_tick(self, event):
        symbol = self.symbols[0]
        if not self.invested:
            df_hist = self._data_board.get_hist_price(symbol, event.timestamp)
            close = df_hist.iloc[-1].Close
            target_size = int(self._position_manager.initial_capital / close)
            self.adjust_position(
                symbol, size_from=0, size_to=target_size, timestamp=event.timestamp
            )
            self.invested = True

Run the strategy

In [ ]:
df = read_data(
    filepath="./test.csv", instrument="37241eecabf1e7223b9db07d2c04fe2c"
)
init_capital = 100_000.0
test_start_date = datetime(
    1990, 1, 1, 8, 30, 0, 0, pytz.timezone("America/New_York")
)
test_end_date = datetime(
    2022, 6, 24, 6, 0, 0, 0, pytz.timezone("America/New_York")
)
strategy = BuyAndHoldStrategy()
strategy.set_capital(init_capital)
strategy.set_symbols(["TTT"])
strategy.set_params(None)

backtest_engine = BacktestEngine(test_start_date, test_end_date)
backtest_engine.set_capital(
    init_capital
)  # capital or portfolio >= capital for one strategy
backtest_engine.add_data("TTT", df)
backtest_engine.set_strategy(strategy)
ds_equity, df_positions, df_trades = backtest_engine.run()

# ------------------------- Evaluation and Plotting -------------------------------------- #
strat_ret = ds_equity.pct_change().dropna()
strat_ret.name = "strat"
bm_ret = strat_ret.copy()
bm_ret.name = "benchmark"
print(strat_ret)

perf_stats_strat = pf.timeseries.perf_stats(strat_ret)
perf_stats_all = perf_stats_strat
perf_stats_bm = pf.timeseries.perf_stats(bm_ret)
perf_stats_all = pd.concat([perf_stats_strat, perf_stats_bm], axis=1)
perf_stats_all.columns = ["Strategy", "Benchmark"]

drawdown_table = pf.timeseries.gen_drawdown_table(strat_ret, 5)
monthly_ret_table = ep.aggregate_returns(strat_ret, "monthly")
monthly_ret_table = monthly_ret_table.unstack().round(3)
ann_ret_df = pd.DataFrame(ep.aggregate_returns(strat_ret, "yearly"))
ann_ret_df = ann_ret_df.unstack().round(3)

print("-------------- PERFORMANCE ----------------")
print(perf_stats_all)
print("-------------- DRAWDOWN ----------------")
print(drawdown_table)
print("-------------- MONTHLY RETURN ----------------")
print(monthly_ret_table)
print("-------------- ANNUAL RETURN ----------------")
print(ann_ret_df)

pf.create_full_tear_sheet(
    strat_ret,
    benchmark_rets=bm_ret,
    positions=df_positions,
    transactions=df_trades,
    round_trips=False,
)
plt.show()